In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Summarize messages

In [2]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import SummarizationMiddleware

agent = create_agent(
    model="gpt-5-nano",
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model="gpt-4o-mini",
            trigger=("tokens", 100),
            keep=("messages", 1)
        )
    ],
)

In [3]:
from langchain.messages import HumanMessage, AIMessage
from pprint import pprint

response = agent.invoke(
    {"messages": [
        HumanMessage(content="What is the capital of the moon?"),
        AIMessage(content="The capital of the moon is Lunapolis."),
        HumanMessage(content="What is the weather in Lunapolis?"),
        AIMessage(content="Skies are clear, with a high of 120C and a low of -100C."),
        HumanMessage(content="How many cheese miners live in Lunapolis?"),
        AIMessage(content="There are 100,000 cheese miners living in Lunapolis."),
        HumanMessage(content="Do you think the cheese miners' union will strike?"),
        AIMessage(content="Yes, because they are unhappy with the new president."),
        HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?"),
        ]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'messages': [HumanMessage(content="Here is a summary of the conversation to date:\n\nThe capital of the moon is Lunapolis. The weather in Lunapolis has clear skies, with a high of 120C and a low of -100C. There are 100,000 cheese miners living in Lunapolis, and there is a possibility that the cheese miners' union will strike due to discontent with the new president.", additional_kwargs={}, response_metadata={}, id='cbff1b74-0238-45ac-b3cc-482f9c585bd1'),
              HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?", additional_kwargs={}, response_metadata={}, id='57cf23a8-654b-4754-95c4-4456a9ed8b21'),
              AIMessage(content='If I were Lunapolis’ new president, I would respond to the cheese miners’ union with an approach that is immediate, fair, and focused on long-term stability. Here’s a practical plan you could adopt or adapt:\n\nCore principles\n- Safety first: mining is dangerous, and climate extremes on the 

In [4]:
print(response["messages"][0].content)

Here is a summary of the conversation to date:

The capital of the moon is Lunapolis. The weather in Lunapolis has clear skies, with a high of 120C and a low of -100C. There are 100,000 cheese miners living in Lunapolis, and there is a possibility that the cheese miners' union will strike due to discontent with the new president.


## Trim/delete messages

In [5]:
from typing import Any
from langchain.agents import AgentState
from langchain.messages import RemoveMessage
from langgraph.runtime import Runtime
from langchain.agents.middleware import before_agent
from langchain.messages import ToolMessage

@before_agent
def trim_messages(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Remove all the tool messages from the state"""
    messages = state["messages"]

    tool_messages = [m for m in messages if isinstance(m, ToolMessage)]
    
    return {"messages": [RemoveMessage(id=m.id) for m in tool_messages]}

In [6]:
agent = create_agent(
    model="gpt-5-nano",
    checkpointer=InMemorySaver(),
    middleware=[trim_messages],
)

In [7]:
response = agent.invoke(
    {"messages": [
        HumanMessage(content="My device won't turn on. What should I do?"),
        ToolMessage(content="blorp-x7 initiating diagnostic ping…", tool_call_id="1"),
        AIMessage(content="Is the device plugged in and turned on?"),
        HumanMessage(content="Yes, it's plugged in and turned on."),
        ToolMessage(content="temp=42C voltage=2.9v … greeble complete.", tool_call_id="2"),
        AIMessage(content="Is the device showing any lights or indicators?"),
        HumanMessage(content="What's the temperature of the device?")
        ]},
    {"configurable": {"thread_id": "2"}}
)

pprint(response)

{'messages': [HumanMessage(content="My device won't turn on. What should I do?", additional_kwargs={}, response_metadata={}, id='8b385708-f9ab-4ff5-bf57-9aeaa4e8c4f3'),
              AIMessage(content='Is the device plugged in and turned on?', additional_kwargs={}, response_metadata={}, id='8b924ec8-7b99-4312-996d-833859d0feaf'),
              HumanMessage(content="Yes, it's plugged in and turned on.", additional_kwargs={}, response_metadata={}, id='f6b1b7f2-ad10-4eb3-b87d-661ab5e2593d'),
              AIMessage(content='Is the device showing any lights or indicators?', additional_kwargs={}, response_metadata={}, id='afd34537-6d73-4484-8501-35d83ddcd144'),
              HumanMessage(content="What's the temperature of the device?", additional_kwargs={}, response_metadata={}, id='d3e1907a-b3a1-48a8-84f0-cd55b9da498e'),
              AIMessage(content='I can’t read your device’s temperature from here. If you want to check it yourself, here are options by device type. Let me know your exac

In [8]:
print(response["messages"][-1].content)

I can’t read your device’s temperature from here. If you want to check it yourself, here are options by device type. Let me know your exact device (brand/model/OS) and I can tailor the steps.

- Computers
  - Windows: Use a tool like HWMonitor, Core Temp, or Open Hardware Monitor. Check CPU/GPU temps. Idle CPU ~30–50°C; under load 60–85°C is typical. If you see temps above ~90°C, shut down and inspect cooling (fans, vents, dust, thermal paste).
  - macOS: Use a third‑party app like iStat Menus or Macs Fan Control to monitor temps. You can also run powermetrics in Terminal for a quick read. Typical temps vary, but don’t let sensors stay very high for long.
  - Linux: Install lm-sensors and run sensors to see CPU/GPU temps.

- Smartphones
  - Android: Some devices show battery/CPU temperature in Settings (Battery) or use a third‑party app (CPU-Z, HWMonitor). Battery temps above ~45–50°C while charging indicate overheating.
  - iPhone: iOS doesn’t always expose temps, but if it’s hot, clo